In [4]:
pip install groq

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 106.5/106.5 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install sqlparse

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

     --------------------------------------- 14.3/14.3 MB 10.7 MB/s eta 0:00:00


In [1]:
import os
from groq import Groq
import sqlparse
import numpy as np
import pandas as pd
import mysql.connector
import json

In [2]:
# Set the environment variable
os.environ['GROQ_API_KEY'] = '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'


In [3]:

def chat_with_groq(client, prompt, model,response_format):
    completion = client.chat.completions.create(
        model=model,
        messages=[
              {
                  "role": "user",
                  "content": prompt
              }
          ],
        )
    response_format=response_format
    return completion.choices[0].message.content



In [4]:

def execute_mysql_query(query):
    conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="admin@123",
            database="device"
        )
    
    try:
        # Execute the query and fetch the results
        cursor = conn.cursor(dictionary=True)  # Use dictionary=True to get results as dicts
        cursor.execute(query)
        query_result = cursor.fetchall()

        # Convert query result to a pandas DataFrame
        results_df = pd.DataFrame(query_result)
    finally:
        # Close the connection
        cursor.close()
        conn.close()

    return results_df



In [5]:
def remove_prefix(text):
    print(text)
    if '{"sql":' in text:
        i=text.index(':')
        text=text[i+1:]
        
    elif '{"error":' in text:
        text=text.replace('{"error":','')
    text=text.replace('}','')
    return text

In [6]:
def get_summarization(client, user_question, df, model):
    """
    This function generates a summarization prompt based on the user's question and the resulting data. 
    It then sends this summarization prompt to the Groq API and retrieves the AI's response.

    Parameters:
    client (Groqcloud): The Groq API client.
    user_question (str): The user's question.
    df (DataFrame): The DataFrame resulting from the SQL query.
    model (str): The AI model to use for the response.

    Returns:
    str: The content of the AI's response to the summarization prompt.
    """
    prompt = '''
      A user asked the following question pertaining to local database tables:
    
      {user_question}
    
      To answer the question, a dataframe was returned:
    
      Dataframe:
      {df}

    In a few sentences, summarize the data in the table as it pertains to the original user question. Avoid qualifiers like "based on the data" and do not comment on the structure or metadata of the table itself
  '''.format(user_question = user_question, df = df)

    # Response format is set to 'None'
    return chat_with_groq(client,prompt,model,None)

In [ ]:



# Use the Llama3 70b model
#model = "llama3-70b-8192"
model = "gemma2-9b-it"

# Get the Groq API key and create a Groq client
groq_api_key = os.getenv('GROQ_API_KEY')
client = Groq(
  api_key=groq_api_key
)

print("Welcome to the MYSQL Query Generator!")
print("You can ask questions about the data in the 'devices' Table.")

# Load the base prompt
with open('prompts/base_prompt.txt', 'r') as file:
  base_prompt = file.read()

while True:
    # Get the user's question
    user_question = input("Ask a question: ")
    if user_question:
        # Generate the full prompt for the AI
        full_prompt = base_prompt.format(user_question=user_question)

        # Get the AI's response as text output
        llm_response = chat_with_groq(client, full_prompt, model,{"type": "json_object"})
        
        print(llm_response)
        result_json = json.loads(llm_response)
        if 'sql' in result_json:
          sql_query = result_json['sql']
          results_df = execute_mysql_query(sql_query)

          formatted_sql_query = sqlparse.format(sql_query, reindent=True, keyword_case='upper')

          print("```sql\n" + formatted_sql_query + "\n```")
          print(results_df.to_markdown(index=False))

          summarization = get_summarization(client,user_question,results_df,model)
          print(summarization)
        elif 'error' in result_json:
          print("ERROR:", 'Could not generate valid SQL for this question')
          print(result_json['error'])
        
        
        
        '''
        print("response is :"+llm_response)
        sql_query = remove_prefix(llm_response)
        print(sql_query)
            
        results_df = execute_mysql_query(sql_query)
        # Format the SQL query for better readability
        formatted_sql_query = sqlparse.format(sql_query, reindent=True, keyword_case='upper')

        
        # Print the SQL query
        print("```sql\n" + formatted_sql_query + "\n```")
        
        # Print the results of the query in markdown format
        print(results_df.to_markdown(index=False))

        # Get summarization
        summarization = get_summarization(client, user_question, results_df, model)
        print(summarization)
    elif 'error' in llm_response.lower():
        # Handle the error case
        print("ERROR: Could not generate valid SQL for this question")
        print(llm_response)
    else:
        # If no SQL or error is found in the response, just print the raw response
        print("AI Response:", llm_response)
        
        '''
        

Welcome to the MYSQL Query Generator!
You can ask questions about the data in the 'devices' Table.
Ask a question: show some laptops which costs below 1000
{"sql": "SELECT name  FROM device.device AS devices WHERE cost < 1000"}
```sql
SELECT name
FROM device.device AS devices
WHERE cost < 1000
```
| name     |
|:---------|
| Laptop A |
| Laptop F |
Two laptops, Laptop A and Laptop F, cost below $1000.  

